In [44]:
import time
import gdown
import pandas as pd
import numpy as np
from datetime import date
import yfinance as yf

pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:,.2f}'.format 

# Question 1

In [2]:
url = "https://stockanalysis.com/ipos/withdrawn/"
df = pd.read_html(url)[0]
df.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [3]:
def create_company_class(df):
    df['company_class_helper'] = df['Company Name'].str.lower()
    # Conditions
    cond = [df['company_class_helper'].str.contains("acquisition|corporation|corp."),
            df['company_class_helper'].str.contains("inc.|incorporated"),
            df['company_class_helper'].str.contains("group"),
            df['company_class_helper'].str.contains("ltd.?|limited"),
            df['company_class_helper'].str.contains("holdings"),]
    choices = ["Acq.Corp", "Inc", "Group", "Limited", "Holdings"]
    df["Company Class"] = np.select(cond, choices, default="Other")
    # Drop helper column
    df.drop('company_class_helper', axis=1, inplace=True)
    return df

In [4]:
def create_avg_price(data):
    df = data.copy()
    df['Price Range'] = df['Price Range'].str.split("-")
    def compute_mean(row):
        try:
            num_list = [float(num.replace("$", "").strip()) for num in row]
            return np.mean(num_list)
        except:
            return None
    df['Avg. Price'] = df['Price Range'].apply(compute_mean)
    return df


In [5]:
df = create_company_class(df)

In [6]:
df = create_avg_price(df)

In [7]:
df.head(10)

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,ODTX,"Odyssey Therapeutics, Inc.","[, ]",-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.","[$3.00 , $4.00]",2000000,Inc,3.50
2,AURN,"Aurion Biotech, Inc.","[, ]",-,Inc,NaN
3,ROTR,"PHI Group, Inc.","[, ]",-,Inc,NaN
4,ONE,One Power Company,"[, ]",-,Other,NaN
5,HPOT,The Great Restaurant Development Holdings Limited,"[$4.00 , $6.00]",1400000,Limited,5.00
6,CABR,"Caring Brands, Inc.",[$4.00],750000,Inc,4.00
7,SQVI,"Sequoia Vaccines, Inc.","[$8.00 , $10.00]",2775000,Inc,9.00
8,SNI,Shenni Holdings Limited,"[$4.00 , $6.00]",3000000,Limited,5.00
9,KMCM,Key Mining Corp.,[$2.25],4444444,Acq.Corp,2.25


In [8]:
df['Shares Offered'] =  pd.to_numeric(df['Shares Offered'], errors='coerce')
df['Withdrawn Value'] = df['Avg. Price'] * df['Shares Offered']

In [9]:
df.notnull().sum()

Symbol             100
Company Name       100
Price Range        100
Shares Offered      72
Company Class      100
Avg. Price          73
Withdrawn Value     71
dtype: int64

In [10]:
agg_df = df.groupby("Company Class", as_index=False)['Withdrawn Value'].sum().sort_values('Withdrawn Value', ascending=False)
agg_df['Withdrawn Value'] = agg_df['Withdrawn Value']/1e6
agg_df

,Company Class,Withdrawn Value
0,Acq.Corp,"4,111.85"
2,Inc,"2,257.16"
4,Other,752.07
3,Limited,549.73
1,Group,33.79


# Question 2

In [11]:
url = "https://stockanalysis.com/ipos/2024/"
df_sharpe = pd.read_html(url)[0]
df_sharpe.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$4.82,3.75%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$23.95,499.25%
2,"Dec 23, 2024",HIT,"Health In Tech, Inc.",$4.00,$0.60,-85.00%
3,"Dec 23, 2024",TDAC,Translational Development Acquisition Corp.,$10.00,$10.26,2.70%
4,"Dec 20, 2024",RANG,Range Capital Acquisition Corp.,$10.00,$10.20,2.00%


In [12]:
df_sharpe['IPO Date'] = pd.to_datetime(df_sharpe['IPO Date'], format='mixed')

In [13]:
df_sharpe_subset = df_sharpe[df_sharpe['IPO Date'] < '2024-06-01'].copy()
df_sharpe_subset.shape, df_sharpe_subset['Symbol'].nunique()

((77, 6), 77)

In [53]:
df_sharpe_subset

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
148,2024-05-31,NAKA,"Kindly MD, Inc.",-,$13.94,-
149,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$36.07,114.65%
150,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$19.37,-0.97%
151,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.53,5.30%
152,2024-05-15,JDZG,JIADE Limited,$4.00,$0.29,-93.00%
153,2024-05-15,RAY,Raytech Holding Limited,$4.00,$1.20,-70.25%
154,2024-05-14,BTOC,Armlogi Holding Corp.,$5.00,$1.66,-69.80%
155,2024-05-10,ZK,ZEEKR Intelligent Technology Holding Limited,$21.00,$25.95,24.33%
156,2024-05-09,GPAT,GP-Act III Acquisition Corp.,$10.00,$10.60,6.00%
157,2024-05-09,PAL,"Proficient Auto Logistics, Inc.",$15.00,$6.98,-53.07%


In [14]:
df_sharpe_subset['IPO Date'].describe()

count                               77
mean     2024-03-17 13:42:51.428571392
min                2024-01-09 00:00:00
25%                2024-02-07 00:00:00
50%                2024-03-22 00:00:00
75%                2024-04-22 00:00:00
max                2024-05-31 00:00:00
Name: IPO Date, dtype: object

In [15]:
tickers = df_sharpe_subset['Symbol'].unique().tolist()

In [16]:

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(tickers):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 NAKA
1 BOW
2 HDL
3 RFAI
4 JDZG
5 RAY
6 BTOC
7 ZK
8 GPAT
9 PAL
10 SVCO
11 NNE
12 CCIX
13 VIK
14 ZONE
15 LOAR
16 MRX
17 RBRK
18 NCI
19 MFI
20 YYGH
21 TRSG
22 CDTG
23 CTRI
24 IBTA
25 MTEN
26 SUPX
27 TWG
28 ULS
29 PACS
30 MNDR
31 CTNM
32 MAMO
33 ZBAO
34 BOLD
35 MMA
36 UBXG
37 IBAC
38 AUNA
39 BKHA
40 LOBO
41 RDDT
42 ALAB
43 INTJ
44 RYDE
45 LGCL
46 SMXT
47 VHAI
48 DYCQ
49 CHRO
50 UMAC
51 HLXB
52 MGX
53 TBBB
54 TELO
55 KYTX
56 PMNT
57 AHR
58 LEGT
59 ANRO
60 GUTS
61 AS
62 FBLG
63 AVBP
64 BTSG
65 HAO
66 CGON
67 YIBO
68 JL
69 SUGP
70 JVSA
71 KSPI
72 CCTG
73 PSBD
74 SYNX
75 SDHC
76 ROMA


In [17]:
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

In [18]:
trading_day = stocks_df[stocks_df['Date']==date(2025, 6, 6)].copy()
trading_day.shape

(77, 27)

In [19]:
trading_day[['Sharpe', 'growth_252d']].notnull().sum()

Sharpe         73
growth_252d    73
dtype: int64

In [20]:
trading_day[['Sharpe', 'growth_252d']].describe()

,Sharpe,growth_252d
count,73.00,73.00
mean,0.28,1.23
std,0.51,1.48
min,-0.08,0.02
25%,0.04,0.29
50%,0.08,0.76
75%,0.29,1.45
max,2.84,8.10


In [21]:
top_10_sharpe = trading_day.sort_values("Sharpe", ascending=False)[['Ticker']].head(10)

In [22]:
top_10_growth_yoy =trading_day.sort_values("growth_252d", ascending=False)[['Ticker']].head(10)

In [23]:
set(top_10_sharpe['Ticker'].values).intersection(set(top_10_growth_yoy['Ticker'].values))

{'JL'}

In [24]:
set(top_10_sharpe['Ticker'].values).difference(set(top_10_growth_yoy['Ticker'].values))

{'BKHA', 'DYCQ', 'HLXB', 'IBAC', 'INTJ', 'JVSA', 'LEGT', 'MNDR', 'TRSG'}

# Question 3

In [25]:
def compute_future_growth(row, days):
    return row.shift(-days) /row

In [26]:
days = [(f"future_growth_{i}m", 21* i) for i in range(1, 13)]

In [27]:
for col, day in days:
    stocks_df[col] = stocks_df.groupby("Ticker")['Close'].transform(lambda x: compute_future_growth(x, day))

In [31]:
selected_cols = ['Close', 'Ticker', 'Year', 'Month', 'Weekday', 'Date'] + [x for x, _ in days]

In [36]:
min_date_df = stocks_df.groupby('Ticker', as_index=False)['Date'].min()

In [38]:
joined_df = stocks_df.merge(min_date_df, on=['Ticker', 'Date'], how='inner')[selected_cols]

In [40]:
joined_df.head()

,Close,Ticker,Year,Month,Weekday,Date,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,3.02,NAKA,2024,5,4,2024-05-31,0.73,0.55,0.40,0.35,0.34,0.39,0.41,0.40,0.59,0.52,0.90,5.67
1,23.80,BOW,2024,5,3,2024-05-23,1.02,1.14,1.30,1.17,1.24,1.46,1.44,1.36,1.39,1.72,1.70,1.53
2,22.29,HDL,2024,5,4,2024-05-17,0.78,0.75,0.67,0.64,0.72,0.72,1.26,1.11,1.03,1.19,0.91,1.00
3,10.04,RFAI,2024,7,4,2024-07-05,1.00,1.01,1.01,1.01,1.01,1.02,1.02,1.03,1.04,1.04,1.05,NaN
4,4.07,JDZG,2024,5,2,2024-05-15,0.24,0.21,0.12,0.16,0.27,0.20,0.14,0.18,0.17,0.15,0.12,0.14


In [42]:
joined_df[[x for x, _ in days]].describe().T

,count,mean,std,min,25%,50%,75%,max
future_growth_1m,77.00,0.93,0.34,0.10,0.78,0.98,1.05,2.65
future_growth_2m,77.00,0.94,0.57,0.07,0.68,1.00,1.14,4.87
future_growth_3m,77.00,0.83,0.41,0.06,0.51,0.93,1.08,2.04
future_growth_4m,77.00,0.82,0.40,0.05,0.52,0.91,1.13,1.61
future_growth_5m,77.00,0.80,0.49,0.05,0.45,0.82,1.02,3.21
future_growth_6m,77.00,0.86,0.65,0.06,0.39,0.80,1.10,3.67
future_growth_7m,77.00,0.85,0.71,0.04,0.32,0.84,1.15,5.12
future_growth_8m,77.00,0.83,0.75,0.04,0.23,0.81,1.07,5.17
future_growth_9m,77.00,0.88,0.93,0.03,0.23,0.82,1.04,6.76
future_growth_10m,76.00,0.91,0.90,0.04,0.25,0.77,1.19,5.35


# Question 4

In [46]:

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=60efadea-2305-427c-8d3d-3c9dd95c5bd8
To: /workspaces/stock-markets-zoomcamp/hw2/data.parquet
100%|██████████| 130M/130M [00:01<00:00, 118MB/s]  


In [48]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

In [50]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

In [51]:
net_income

np.float64(24295.523125248386)